## Comparison of Toront, New York City, and Beijing

In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# !pip install geopy
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
# !pip install folium
import folium
import json 
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize 

### Import Datasets

#### Import Toronto data

In [2]:
#get table
url_tor = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url_tor)
data = BeautifulSoup(page, "lxml")

In [3]:
all_tables=data.find_all("table")
all_tables

[<table class="wikitable sortable">
 <tbody><tr>
 <th>Postal Code
 </th>
 <th>Borough
 </th>
 <th>Neighborhood
 </th></tr>
 <tr>
 <td>M1A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M2A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M3A
 </td>
 <td>North York
 </td>
 <td>Parkwoods
 </td></tr>
 <tr>
 <td>M4A
 </td>
 <td>North York
 </td>
 <td>Victoria Village
 </td></tr>
 <tr>
 <td>M5A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Regent Park, Harbourfront
 </td></tr>
 <tr>
 <td>M6A
 </td>
 <td>North York
 </td>
 <td>Lawrence Manor, Lawrence Heights
 </td></tr>
 <tr>
 <td>M7A
 </td>
 <td>Downtown Toronto
 </td>
 <td>Queen's Park, Ontario Provincial Government
 </td></tr>
 <tr>
 <td>M8A
 </td>
 <td>Not assigned
 </td>
 <td>Not assigned
 </td></tr>
 <tr>
 <td>M9A
 </td>
 <td>Etobicoke
 </td>
 <td>Islington Avenue, Humber Valley Village
 </td></tr>
 <tr>
 <td>M1B
 </td>
 <td>Scarborough
 </td>
 <td>Malvern, Rouge
 </td></tr>
 <tr>
 <td>M2B


In [4]:
right_table=data.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [5]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

df.head(5)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#clean data
indexnames = df[df['Borough'].str.contains('Not assigned')].index
df.drop(indexnames, inplace = True)

In [7]:
df = df.reset_index(drop=True)
df = df.replace('\n','', regex=True)
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [11]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#merge two datasets
merged_left = pd.merge(df, df_data_1, on='Postal Code')
merged_left.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [14]:
#Downtown Toronto is Downtown Toronto
Toronto_data = merged_left[merged_left['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [15]:
print('The dataframe has {} neighborhoods.'.format(
        len(Toronto_data['Neighborhood'].unique())))

The dataframe has 19 neighborhoods.


#### Import NYC

In [16]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [17]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']
neighborhoods_data 

[{'type': 'Feature',
  'id': 'nyu_2451_34572.1',
  'geometry': {'type': 'Point',
   'coordinates': [-73.84720052054902, 40.89470517661]},
  'geometry_name': 'geom',
  'properties': {'name': 'Wakefield',
   'stacked': 1,
   'annoline1': 'Wakefield',
   'annoline2': None,
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.84720052054902,
    40.89470517661,
    -73.84720052054902,
    40.89470517661]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.2',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82993910812398, 40.87429419303012]},
  'geometry_name': 'geom',
  'properties': {'name': 'Co-op City',
   'stacked': 2,
   'annoline1': 'Co-op',
   'annoline2': 'City',
   'annoline3': None,
   'annoangle': 0.0,
   'borough': 'Bronx',
   'bbox': [-73.82993910812398,
    40.87429419303012,
    -73.82993910812398,
    40.87429419303012]}},
 {'type': 'Feature',
  'id': 'nyu_2451_34572.3',
  'geometry': {'type': 'Point',
   'coordinates': [-73.82780644716412, 

In [18]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [19]:
#Downtown NYC is Manhattan
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [20]:
print('The dataframe has {} neighborhoods.'.format(
        len(manhattan_data['Neighborhood'].unique())))

The dataframe has 40 neighborhoods.


#### Import Beijing data

In [21]:
body = client_1bab0aeef8cc4aba968a7303fbd8954c.get_object(Bucket='capstoneproject-donotdelete-pr-4hgq4g1jminbx3',Key='Beijing_District.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

beijing = pd.read_csv(body)
beijing['District'].unique()

array(['Changping', 'Chaoyang', 'Daxing', 'Dongcheng', 'Fangshan',
       'Fengtai', 'Haidian', 'Huairou', 'Mentougou', 'Pinggu',
       'Shijingshan', 'Shunyi', 'Tongzhou', 'Xicheng', 'Miyun', 'Yanqing'],
      dtype=object)

In [22]:
beijing.rename(columns={'District':'Borough','Subdisctricts':'Neighborhood'}, inplace=True)
beijing.head()
#Beijing downtown is Dongcheng and Xicheng
Beijing_district = beijing[beijing['Borough'].str.contains('cheng') ].reset_index(drop=True)
Beijing_district.head()

,City,Borough,Neighborhood
0,Beijing,Dongcheng,Jingshan
1,Beijing,Dongcheng,Donghuamen
2,Beijing,Dongcheng,Jiaodaokou
3,Beijing,Dongcheng,Andingmen
4,Beijing,Dongcheng,Beixinqiao


In [24]:
#create neighbourhoods information
bj_neighborhoods= pd.DataFrame(columns = ['City','Borough','Neighborhood'])

for ii in range(Beijing_district.shape[0]):
    borough = Beijing_district.loc[ii,'Borough']
    neighborhood = Beijing_district.loc[ii,'Neighborhood']
      
    #find the location data, ignore the neighborhoods that are unable to be located by Nominatim
    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode("{},{},Beijing,China".format(neighborhood,borough))
    
    #try one more searching
    if location is None: 
        location = geolocator.geocode("{},Beijing,China".format(neighborhood))
        
    if location is None: 
        print("The location data of {} in {} is not available!".format(neighborhood,borough))
    else:
        bj_neighborhoods = bj_neighborhoods.append({'City': 'Beijing',
                                                'Borough': borough,
                                                'Neighborhood': neighborhood,
                                                'Latitude': location.latitude,
                                                'Longitude': location.longitude
                                               }, ignore_index=True)

#only 6 locations don't have cooridinates, but it doesn't influence the analysis

The location data of Dingsi in Dongcheng is not available!
The location data of Changwenmenwai in Dongcheng is not available!
The location data of Dasheng in Xicheng is not available!
The location data of Dazhalan in Xicheng is not available!
The location data of Chunshu in Xicheng is not available!
The location data of Baizhifang in Xicheng is not available!


In [25]:
bj_neighborhoods.head()
bj_neighborhoods = bj_neighborhoods.drop('City', axis = 1)

In [27]:
bj_neighborhoods['Borough'] = 'Downtown Beijing'
beijing_data = bj_neighborhoods.copy()
beijing_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Beijing,Jingshan,39.927006,116.392088
1,Downtown Beijing,Donghuamen,39.913594,116.395206
2,Downtown Beijing,Jiaodaokou,39.937807,116.402343
3,Downtown Beijing,Andingmen,39.947729,116.402067
4,Downtown Beijing,Beixinqiao,39.939383,116.410870


In [28]:
print('The dataframe has {} neighborhoods.'.format(
        len(beijing_data['Neighborhood'].unique())))

The dataframe has 26 neighborhoods.


### ADD Venue information for 3 cities

In [29]:
#use Foursquare
CLIENT_ID = 'J0M4N0VALCKPI20001ZXUN5H1EY3WF52SXMZ10PB5C4QLCDA' 
CLIENT_SECRET = 'KS0UKP4UVZJOT53VETCODUHUBY1ST5A3QU0JNHRAHVONXIBI' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

LIMIT = 100
radius = 500

Your credentails:
CLIENT_ID: J0M4N0VALCKPI20001ZXUN5H1EY3WF52SXMZ10PB5C4QLCDA
CLIENT_SECRET:KS0UKP4UVZJOT53VETCODUHUBY1ST5A3QU0JNHRAHVONXIBI


In [30]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
#define a function of get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
def getvenuesdetails(city):
    city.loc[0, 'Neighborhood']
    neighborhood_latitude = city.loc[0, 'Latitude'] 
    neighborhood_longitude = city.loc[0, 'Longitude'] 
    
    url =  'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

    neighborhood_name = city.loc[0, 'Neighborhood'] 

    results = requests.get(url).json()

    venues = results['response']['groups'][0]['items']

    nearby_venues = json_normalize(venues) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues = nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    city_venues = getNearbyVenues(names=city['Neighborhood'],
                                       latitudes=city['Latitude'],
                                       longitudes=city['Longitude'])
    print(city_venues.shape)
    print(city_venues.groupby('Neighborhood').count())
    print('There are {} uniques categories.'.format(len(city_venues['Venue Category'].unique())))
    
    return city_venues

In [33]:
Toronto_venue = getvenuesdetails(Toronto_data)
Toronto_venue

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley
(1218, 7)
                                                    Neighborhood Latitude  \
Neighborhood                                                                
Berczy Park                                                            56   
CN Tower, King and Spadina, Railway Lands, Harb...                     16   
Central Bay Street                                                     64   
Christie        

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
5,"Regent Park, Harbourfront",43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
6,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
7,"Regent Park, Harbourfront",43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
8,"Regent Park, Harbourfront",43.654260,-79.360636,The Extension Room,43.653313,-79.359725,Gym / Fitness Center
9,"Regent Park, Harbourfront",43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [34]:
NYC_venue = getvenuesdetails(manhattan_data)
NYC_venue

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
(3140, 7)
                     Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                                
Battery Park City                       63                      63     63   
Carnegie Hill                           88                      88     88   
Central Harlem                          46                      46     46   
Chelsea                                100                     100    100   
Chinatown   

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.910660,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.910660,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.910660,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.910660,Dunkin',40.877136,-73.906666,Donut Shop
5,Marble Hill,40.876551,-73.910660,Rite Aid,40.875467,-73.908906,Pharmacy
6,Marble Hill,40.876551,-73.910660,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
7,Marble Hill,40.876551,-73.910660,Blink Fitness,40.877271,-73.905595,Gym
8,Marble Hill,40.876551,-73.910660,Starbucks,40.873755,-73.908613,Coffee Shop
9,Marble Hill,40.876551,-73.910660,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium


In [35]:
Beijing_venue = getvenuesdetails(beijing_data)
Beijing_venue

Jingshan
Donghuamen
Jiaodaokou
Andingmen
Beixinqiao
Chaoyangmen
Jianguomen
Dongzhimen
Hepingli
Qianmen
Donghuashi
Longtan
Tiyuguan
Tiantan
Yongdingmenwai
Jinrong
West Chang'an
Xinjiekou
Yuetan
Zhanlan Road
Shichahai
Tianqiao
Taoranting
Guang'anmennei
Niujie
Guang'anmenwai
(362, 7)
                Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                           
Andingmen                          23                      23     23   
Beixinqiao                         25                      25     25   
Chaoyangmen                        17                      17     17   
Donghuamen                         23                      23     23   
Donghuashi                          4                       4      4   
Dongzhimen                         31                      31     31   
Guang'anmennei                      9                       9      9   
Guang'anmenwai                      4                       4      4  

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jingshan,39.927006,116.392088,Jingshan Park (景山公园),39.923536,116.390576,Park
1,Jingshan,39.927006,116.392088,Wanchun Pavilion (万春亭),39.923489,116.390420,Historic Site
2,Jingshan,39.927006,116.392088,City Walls Courtyard House Beijing,39.928131,116.392442,Hostel
3,Jingshan,39.927006,116.392088,TRB Hutong,39.926528,116.397204,French Restaurant
4,Jingshan,39.927006,116.392088,The Central Point of Beijing City (北京城中心点),39.923456,116.390407,Scenic Lookout
5,Jingshan,39.927006,116.392088,Alley Cafe,39.923973,116.393513,Café
6,Jingshan,39.927006,116.392088,Man Fu Lou (满福楼大酒楼),39.928046,116.390397,Hotpot Restaurant
7,Jingshan,39.927006,116.392088,皇家冰窖小院,39.928844,116.387190,Chinese Restaurant
8,Jingshan,39.927006,116.392088,Dongzhimen Train Station Metro Station,39.925579,116.395010,Metro Station
9,Jingshan,39.927006,116.392088,Shatan Hotel Beijing 北京沙滩宾馆,39.924032,116.394973,Hotel


In [ ]:
#plot maps!!!

In [371]:
# one hot encoding
merge_onehot = pd.get_dummies(df['Borough'], prefix="", prefix_sep="")
merge_onehot['Neighborhood'] = df['Neighborhood'] 
merge_onehot['Postal Code'] = df['Postal Code'] 
merge_onehot
fixed_columns = [merge_onehot.columns[-1]] + list(merge_onehot.columns[:-1])
merge_onehot = merge_onehot[fixed_columns]
merge_onehot.head()

,Postal Code,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Scarborough,West Toronto,York,Neighborhood
0,M3A,0,0,0,0,0,0,1,0,0,0,Parkwoods
1,M4A,0,0,0,0,0,0,1,0,0,0,Victoria Village
2,M5A,0,1,0,0,0,0,0,0,0,0,"Regent Park, Harbourfront"
3,M6A,0,0,0,0,0,0,1,0,0,0,"Lawrence Manor, Lawrence Heights"
4,M7A,0,1,0,0,0,0,0,0,0,0,"Queen's Park, Ontario Provincial Government"


In [372]:
merge_onehot.shape

(103, 12)

In [373]:
merge_grouped = merge_onehot.groupby('Neighborhood').mean().reset_index()
merge_grouped

,Neighborhood,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Scarborough,West Toronto,York
0,Agincourt,0,0,0,0,0,0,0,1,0,0
1,"Alderwood, Long Branch",0,0,0,0,1,0,0,0,0,0
2,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,1,0,0,0
3,Bayview Village,0,0,0,0,0,0,1,0,0,0
4,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,1,0,0,0
5,Berczy Park,0,1,0,0,0,0,0,0,0,0
6,"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,1,0,0
7,"Brockton, Parkdale Village, Exhibition Place",0,0,0,0,0,0,0,0,1,0
8,"Business reply mail Processing Centre, South C...",0,0,1,0,0,0,0,0,0,0
9,"CN Tower, King and Spadina, Railway Lands, Har...",0,1,0,0,0,0,0,0,0,0


In [374]:
merge_grouped.shape

(99, 11)

In [375]:
num_top_venues = 5

for hood in merge_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = merge_grouped[merge_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))

----Agincourt----
              venue  freq
0       Scarborough   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Alderwood, Long Branch----
              venue  freq
0         Etobicoke   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Bathurst Manor, Wilson Heights, Downsview North----
              venue  freq
0        North York   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Bayview Village----
              venue  freq
0        North York   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Bedford Park, Lawrence Manor East----
              venue  freq
0        North York   1.0
1   Central Toronto   0.0
2  Downtown Toronto   0.0
3      East Toronto   0.0
4         East York   0.0
----Berczy Park----
              venue  freq
0  

In [376]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [378]:
num_top_venues = 8

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = merge_grouped['Neighborhood']

for ind in np.arange(merge_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(merge_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Agincourt,Scarborough,York,West Toronto,North York,Mississauga,Etobicoke,East York,East Toronto
1,"Alderwood, Long Branch",Etobicoke,York,West Toronto,Scarborough,North York,Mississauga,East York,East Toronto
2,"Bathurst Manor, Wilson Heights, Downsview North",North York,York,West Toronto,Scarborough,Mississauga,Etobicoke,East York,East Toronto
3,Bayview Village,North York,York,West Toronto,Scarborough,Mississauga,Etobicoke,East York,East Toronto
4,"Bedford Park, Lawrence Manor East",North York,York,West Toronto,Scarborough,Mississauga,Etobicoke,East York,East Toronto


In [379]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 4

merge_grouped_clustering = merge_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(merge_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 1, 1, 1, 3, 2, 0, 0, 3], dtype=int32)

In [382]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = merged_data

merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York


In [383]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [384]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
4,East Toronto,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
9,West Toronto,0,West Toronto,York,Scarborough,North York,Mississauga,Etobicoke,East York,East Toronto
11,West Toronto,0,West Toronto,York,Scarborough,North York,Mississauga,Etobicoke,East York,East Toronto
12,East Toronto,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
14,West Toronto,0,West Toronto,York,Scarborough,North York,Mississauga,Etobicoke,East York,East Toronto
15,East Toronto,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
17,East Toronto,0,East Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
18,Central Toronto,0,Central Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
19,Central Toronto,0,Central Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
20,Central Toronto,0,Central Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York


In [385]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue


In [386]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue


In [387]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[1] + list(range(5, merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue
0,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
1,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
2,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
3,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
5,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
6,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
7,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
8,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
10,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
13,Downtown Toronto,3,Downtown Toronto,York,West Toronto,Scarborough,North York,Mississauga,Etobicoke,East York
